(overview_stats)=
# Overview


ArviZ-stats is the computing library of ArviZ. And it can be used through two interfaces

* The xarray interface, which is used to compute statistics on xarray objects.
* The array interface, which is used to compute statistics on numpy arrays.

The array interface is low-level and has minimal dependencies, it is intended for advanced users and developers of third-party libraries. In contrast, the xarray interface is designed for end users, which simplifies usage by automating common tasks and handling metadata. In this overview, we will focus on the xarray interface. For a description of the array interface, please refer to the {ref}`array interface <array_interface>` guide.

## Getting Started with ArviZ-stats

To start, we’ll import ArviZ.


In [1]:
import arviz_base as azb

import arviz_stats as azs

To make the example easier to follow we are going to skip the modeling step, and assume we have already built and solved a Bayesian model. ArviZ provides several pre-saved models, so we can use one of those to demonstrate the statistical functionality. In this case we will use the `centered_eight` model, which is a classic, but even if you are not familiar with it, you can still follow along and see how the computations works.

In [2]:
data = azb.load_arviz_data("centered_eight")

A common task in Bayesian data analysis is to compute posterior summaries. For instance, we can use the `summary` function to summarize the posterior distribution of a model. By default this function computes a series of statistics, including the mean, standard deviation, and credible intervals (defaults to equally tailed interval `eti`) for each variable in the model. It also computes a series of diagnostics, including the effective sample size (ess), the rhat statistic, and the Monte Carlo standard error (mcse).

In [3]:
azs.summary(data)

,mean,sd,eti89_lb,eti89_ub,ess_bulk,ess_tail,r_hat,mcse_mean,mcse_sd
mu,4.49,3.49,-1.08,9.85,240.99,370.29,1.02,0.23,0.16
theta[Choate],6.46,5.87,-1.86,16.13,365.05,503.61,1.01,0.30,0.35
theta[Deerfield],5.03,4.88,-2.66,12.55,427.32,525.26,1.01,0.23,0.18
theta[Phillips Andover],3.94,5.69,-5.06,12.20,514.72,702.23,1.01,0.23,0.24
theta[Phillips Exeter],4.87,5.01,-3.06,12.47,337.18,552.70,1.01,0.26,0.22
theta[Hotchkiss],3.67,4.95,-4.70,10.94,365.35,482.91,1.01,0.25,0.20
theta[Lawrenceville],3.97,5.19,-4.47,11.54,521.46,675.27,1.01,0.22,0.21
theta[St. Paul's],6.58,5.10,-1.12,15.32,275.68,352.82,1.01,0.30,0.25
theta[Mt. Hermon],4.77,5.74,-3.95,13.53,451.86,630.26,1.01,0.26,0.27
tau,4.12,3.10,1.09,9.83,66.57,58.22,1.06,0.26,0.33


 In this example, there are 3 variables (or parameters), `mu`, `tau` and `theta`, the first two are unidimensional, while `theta` is a vector of length 8. Thus, in total we got 10 rows. If we want to focus on a specific variable or a subset of variables, we can use the `var_names` argument to specify which ones to display. Simply pass the names of the variables you'd like to summarize.

In [4]:
azs.summary(data, var_names=["mu", "tau"])

,mean,sd,eti89_lb,eti89_ub,ess_bulk,ess_tail,r_hat,mcse_mean,mcse_sd
mu,4.49,3.49,-1.08,9.85,240.99,370.29,1.02,0.23,0.16
tau,4.12,3.10,1.09,9.83,66.57,58.22,1.06,0.26,0.33


or negate those for which we do not want to calculate summaries.

In [5]:
azs.summary(data, var_names=["~theta"])

,mean,sd,eti89_lb,eti89_ub,ess_bulk,ess_tail,r_hat,mcse_mean,mcse_sd
mu,4.49,3.49,-1.08,9.85,240.99,370.29,1.02,0.23,0.16
tau,4.12,3.10,1.09,9.83,66.57,58.22,1.06,0.26,0.33


Let's say that we now want to do something else like computing the root mean square deviation between the model predictions and the observed data. We can use the `metrics` function to compute this statistic.

In [6]:
azs.metrics(data)

rmse(mean=9.6, se=2.6)

## ArviZ and DataTree

So far we have seen two functions, they both use the same input `data`, but generate different results. This is a common pattern in ArviZ, we pass an object like `data` and ArviZ performs different computations based on the function we call. This allows us to easily switch between different functions without having to change the underlying data structure. Internally ArviZ selects the necessary subset of the data to perform the computations. Thus, we don't need to worry about the details of how the data is organized or how to extract the relevant information for each function, unless we want to.

In this example, `data` is a `DataTree` object; a flexible and efficient structure for storing and manipulating data in ArviZ. While DataTrees are used extensively in ArviZ, they are not specific to it; they are actually a core data structure in the [xarray](https://docs.xarray.dev/en/stable/) library. We rely on DataTrees because they offer a powerful way to represent multi-dimensional data with labeled axes, making it straightforward to work with the complex datasets, including those generated during a Bayesian workflow, such as posterior samples, prior samples, and sampling statistics. This structure promotes clear organization and simplifies both data access and manipulation.

To keep this introduction short and simple we are going to focus our attention on some very general concepts related to DataTrees in the context of ArviZ. Details on how to work with DataTrees, and a description of other data-structures used by ArviZ, can be found in the [Working With DataTree](https://arviz-devs.github.io/EABM/Chapters/DataTree.html) guide. 

At a high level, a `DataTree` is a hierarchical structure with groups, the number of groups varies but within ArviZ we have a convention for the names of the groups. For instance, the posterior samples are stored in a group called `posterior`, the prior samples are stored in a group called `prior`, and the sampling statistics (stuff related to the inner workings of samplers) are stored in a group called `sample_stats`. This allows for a clear organization of the data, making it easy to access and manipulate.

We can see that for the `centered_eight` model we have several groups:

In [7]:
data.groups

('/',
 '/posterior',
 '/posterior_predictive',
 '/log_likelihood',
 '/sample_stats',
 '/prior',
 '/prior_predictive',
 '/observed_data',
 '/constant_data')

Each group can contain multiple variables. We already saw that we have `mu`, `theta`, and `tau`. And each variable can have multiple dimensions. For instance, the variable `mu` has a dimension called `chain`, which represents the different chains used in the MCMC sampling process, and a dimension called `draw`, which represents the different samples drawn from the posterior distribution. `theta` has these two dimensions plus a dimension called `school`, which represents each one of the 8 schools in the model.

In [8]:
data.posterior.data_vars

Data variables:
    mu       (chain, draw) float64 16kB 7.872 3.385 9.1 ... 1.767 3.486 3.404
    theta    (chain, draw, school) float64 128kB 12.32 9.905 ... 6.762 1.295
    tau      (chain, draw) float64 16kB 4.726 3.909 4.844 ... 2.741 2.932 4.461

This is a more complete view of the content of the posterior group

In [9]:
data.posterior

<xarray.DataTree 'posterior'>
Group: /posterior
    Dimensions:  (chain: 4, draw: 500, school: 8)
    Coordinates:
      * chain    (chain) int64 32B 0 1 2 3
      * draw     (draw) int64 4kB 0 1 2 3 4 5 6 7 ... 493 494 495 496 497 498 499
      * school   (school) <U16 512B 'Choate' 'Deerfield' ... 'Mt. Hermon'
    Data variables:
        mu       (chain, draw) float64 16kB 7.872 3.385 9.1 ... 1.767 3.486 3.404
        theta    (chain, draw, school) float64 128kB 12.32 9.905 ... 6.762 1.295
        tau      (chain, draw) float64 16kB 4.726 3.909 4.844 ... 2.741 2.932 4.461
    Attributes:
        created_at:                 2022-10-13T14:37:37.315398
        arviz_version:              0.13.0.dev0
        inference_library:          pymc
        inference_library_version:  4.2.2
        sampling_time:              7.480114936828613
        tuning_steps:               1000

### Exploring groups, dimensions and coordinates

By default the xarray interface will use one or more of the available groups. For instance the `summary` function will use the `posterior` group, while the `metrics` function will use both the `posterior_predictive` and the `observed_data` groups. These two functions combine the `chain` and `draw` dimensions to produce the final computations. That explains, for example, why we get a single mean for each variable when calling summary even when the posterior is represented using 2000 samples, 4 chains x 500 draws. But notice, for instance, that the dimension `school` is not "reduced", instead we get a different "mean" (and other quantities) for each school.

All these defaults can be changed, as we already saw with the `var_names` argument. Usually ArviZ functions will have a `group` argument. For instance the `metrics` function does not have it, as computations are always done using the `posterior_predictive` and the `observed_data`, but summary has it and allows you to specify many different groups.


Let's set group to `posterior_predictive`. Can you anticipate what we will get?

In [10]:
azs.summary(data, group="posterior_predictive")

,mean,sd,eti89_lb,eti89_ub,ess_bulk,ess_tail,r_hat,mcse_mean,mcse_sd
obs[Choate],6.65,15.86,-18.52,32.25,1370.99,1595.61,1.00,0.43,0.32
obs[Deerfield],4.88,11.25,-13.05,22.62,1279.14,1441.16,1.00,0.31,0.22
obs[Phillips Andover],4.26,16.42,-21.52,31.11,1673.19,1451.91,1.00,0.40,0.28
obs[Phillips Exeter],5.00,12.27,-14.15,24.40,1646.09,1751.54,1.00,0.30,0.22
obs[Hotchkiss],3.46,9.93,-11.94,19.03,1600.17,1801.21,1.01,0.25,0.18
obs[Lawrenceville],3.42,12.22,-16.47,22.05,1537.25,1707.15,1.00,0.31,0.22
obs[St. Paul's],6.36,11.16,-11.84,24.07,1440.17,1553.88,1.00,0.29,0.20
obs[Mt. Hermon],4.92,18.90,-25.17,35.16,1948.58,1935.88,1.00,0.43,0.31


We get predictions of the model at each observation. In this example, we have 8 observations, so we get 8 summaries.

ArviZ does not come with guarantees that any possible combination of arguments that you may want to try will work, or that the generated result will be sensible or useful. Defaults usually are, but then you are free to explore the data as you wish and be responsible for the results.


For instance

```python
azs.summary(data, group="sample_stats")
```

will not work, because the `sample_stats` group does not contain samples to summarize. However,

```python
azs.summary(data, group="log_likelihood")
```

will work, and it will compute the summary for the log-likelihood per observation. How useful this is depends on the context of the analysis.

### Sample_dims

Now, we are going to discuss the `sample_dims` argument. This argument is used to specify which dimensions of the data should be reduced when doing calculations. An example will help us understand this better. Let's say we want to use the `summary` function to summarize the observed data. If we do:

```python
azs.summary(data, group="observed_data")
```

We will get an error, instead we should write:

In [11]:
azs.summary(data, group="observed_data", sample_dims=["school"], kind="stats")

,mean,sd,eti89_lb,eti89_ub
obs,8.75,9.77,-2.23,24.15


We need to add these two arguments for two separate reasons.

* By default, `sample_dims` is set to `["chain", "draw"]`, but those dimensions are not present (or meaningful) in the `observed_data` group. By passing `sample_dims=["school"]`, we tell ArviZ to reduce the `school` dimension, which is the only dimension present in the `observed_data` group. 

* Conceptually it does not make sense to compute sampling diagnostics on observed data, as they are not samples generated using MCMC methods. Additionally, we will run into issues if we try to compute the `r_hat` diagnostics, as it assumes we have at least two chains.

If we were interested in summarizing the posterior distribution for `mu` per `chain`, we could do:

In [12]:
azs.summary(data,
            var_names=["mu"],
            kind="stats",
            sample_dims=["draw"])

,mean,sd,eti89_lb,eti89_ub
mu[0],4.25,3.40,-0.97,9.70
mu[1],4.18,3.24,-0.94,9.46
mu[2],4.66,3.81,-1.29,10.30
mu[3],4.85,3.43,-1.10,9.84


We can compute the summary for `sample_dims=["draw", "school"]`

In [13]:
azs.summary(data,
            kind="stats",
            sample_dims=["draw", "school"])

,mean,sd,eti89_lb,eti89_ub
mu[0],4.25,3.40,-0.97,9.70
mu[1],4.18,3.24,-0.94,9.46
mu[2],4.66,3.81,-1.29,10.30
mu[3],4.85,3.43,-1.10,9.84
theta[0],4.63,5.04,-3.30,12.21
theta[1],4.64,5.31,-3.58,13.20
theta[2],5.17,5.87,-3.88,13.99
theta[3],5.21,5.38,-2.90,13.18
tau[0],3.68,2.71,1.14,8.81
tau[1],4.25,3.15,0.90,9.85


Notice that `draw` is present in the 3 variables, but `school` is only present in `theta`. Thus, we get a summary for `mu` and `tau` that is reduced over the `draw` dimension, while for `theta` we get a summary that is reduced over both `draw` and `school`.

## Coords

We can use the `coords` argument to specify which **coordinates** we want to use for the computations. This is useful to filter variables across dimensions. For instance, if we want to summarize the posterior distribution of only a subset of schools, we can do:

In [14]:
azs.summary(data,
              var_names=["mu", "theta"],
              coords={"school": ["Choate", "Deerfield"]},
              )

,mean,sd,eti89_lb,eti89_ub,ess_bulk,ess_tail,r_hat,mcse_mean,mcse_sd
mu,4.49,3.49,-1.08,9.85,240.99,370.29,1.02,0.23,0.16
theta[Choate],6.46,5.87,-1.86,16.13,365.05,503.61,1.01,0.30,0.35
theta[Deerfield],5.03,4.88,-2.66,12.55,427.32,525.26,1.01,0.23,0.18


Notice that the `coords` argument is used to select specific schools and thus it does not affect variables without the school dimension, like `mu`.

## Global defaults

ArviZ has a set of defaults that apply globally, meaning that you can change them in one place and it will affect the behaviour of many functions. ArviZ's functions, like `summary`, take from here the default value for `sample_dims` and other arguments. They are stored in `azb.rcParams`. Can you spot the default values for interval and their probability?

In [15]:
azb.rcParams

RcParams({'data.http_protocol': 'https',
          'data.index_origin': 0,
          'data.sample_dims': ('chain', 'draw'),
          'data.save_warmup': False,
          'plot.backend': 'matplotlib',
          'plot.density_kind': 'kde',
          'plot.max_subplots': 40,
          'stats.ci_kind': 'eti',
          'stats.ci_prob': 0.89,
          'stats.ic_compare_method': 'stacking',
          'stats.ic_pointwise': True,
          'stats.ic_scale': 'log',
          'stats.module': 'base',
          'stats.point_estimate': 'mean'})

If we don't like the default value of 0.89 for the credible interval, we can change it globally:

```python
azb.rcParams["stats.ci_prob"] = 0.95
```

This will change the default value for the credible interval to 0.95 for all subsequent calls to `summary` and other functions that use this parameter, or we can change it per call:

```python
azs.summary(data, ci_prob=0.95)
```

:::{seealso}
* [arviz-base](https://arviz-base.readthedocs.io/en/latest/) data related functionality, including converters from different PPLs.
* [arviz-plots](https://arviz-plots.readthedocs.io/en/latest/) for visual checks built on top of arviz-stats and arviz-base.
* [Exploratory Analysis of Bayesian Models](https://arviz-devs.github.io/EABM/) online book. 
* The {ref}`array interface <array_interface>` tutorial, for a description of the array interface.
:::